In [9]:
'''Outputs a df of words that appear in the reviews sorted by counts, excluding many common words.'''

#directory='c:/users/da1933/desktop/cds/ids_project/'
directory='~/desktop/idsproject/'

import pandas as pd
from collections import Counter
import numpy as np

# Load and combine datasets
df1=pd.read_csv(directory+'wine/winemag-data-130k-v2.csv',index_col=0)
df2=pd.read_csv(directory+'wine/winemag-data_first150k.csv',index_col=0)
df=pd.concat([df1.drop(['taster_name','taster_twitter_handle','title'],axis=1),df2]).reset_index(drop=True)

# Create dictionary to store counter objects
cdict={i:[] for i in range(int(np.round(df.shape[0]/1000)))}

# counters are created in batches of size 1000 because looping through
# entire df is very slow.

start=0
target=1000

for i in cdict.keys():
    counter=Counter()
    for description in df['description'][start:target]:
        new_counter=Counter(description.replace('.','').replace(',','').lower().split())
        counter=counter+new_counter
    start+=1000
    target+=1000
    cdict[i].append(counter)

bigcounter=Counter()

# Combine individual counters and convert to df
for i in cdict.keys():
    bigcounter=bigcounter+cdict[i][0]
all_counts=pd.DataFrame.from_dict(dict(bigcounter),orient='index').sort_values(by=0,ascending=False).reset_index()

# Remove some of the most common words
common=['and','the','a','of','with','this','is','wine','in','it','to','on','that','from','but',"it's",
        'are','has','for','by','as','an','its','now','offers','while','shows','through','well','at','very',
        'more','some','will','not','out','up','or','be','all','made','also','note','just','into','there',
        'like','bit','still','long','give',"there's",'so','alongside','along','over','ready','which','one',
        'yet','than','time','almost','little','you','intense','come']
all_counts=all_counts[~all_counts['index'].isin(common)]

all_counts.to_csv('word_counts.csv',index=False)